In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using LearningMPC
using Gurobi
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

In [ ]:
include("cartpoles.jl")

In [ ]:
cartpole = CartPoles.CartPole()
mechanism = cartpole.mechanism
basevis = Visualizer()[:cartpole]
delete!(basevis)
setgeometry!(basevis, cartpole)

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "slider_to_cart"), [0])
set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [0.1])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
u0 = zeros(num_velocities(x0))
xstar = MechanismState(mechanism, q0, v0)

params = CartPoles.CartPoleMPCParams(Δt = 0.02)

In [ ]:
controller = CartPoles.MPCController(cartpole, params, xstar);
controller.callback = (x, results) -> begin
    if !isnull(results.lcp_updates)
        playback(basevis[:robot], get(results.lcp_updates), controller.params.Δt)
    end
end
    

In [ ]:
results_mpc = LCPSim.simulate(x0, controller, cartpole.environment, params.Δt, 20, GurobiSolver(OutputFlag=0));

Data to collect:

* current state
* first input
* input/state jacobian
* solve time
* optimality gap
* solution status

In [ ]:
fname = "cart-pole-data-0.02.jld2"
records = jldopen(fname, "a+") do file
    haskey(file, "records") ? file["records"] : Tuple{Vector{Float64}, Vector{Float64}, Matrix{Float64}}[]
end